In [ ]:
import pandas as pd

In [ ]:
all_decisions_df = pd.read_csv("results/all_decisions_df.csv")
all_decisions_df.fillna({"model": ""}, inplace=True)

In [ ]:
decision_counts = pd.pivot(
    all_decisions_df.query("task_scope != 'n-gram'").groupby(
        ["model", "task_scope", "dataset", "experiment_run", "decision"]
    )["benchmark"].count().reset_index(),
    index=["dataset", "model", "task_scope", "experiment_run"],
    columns="decision",
    values=["benchmark"],
).fillna(0).reset_index()

decision_counts["decisiveness"] = (
    decision_counts.loc[:, (slice(None), ["no", "yes"])].sum(axis="columns") /
    decision_counts.loc[:, (slice(None), ["no", "unknown", "yes"])].sum(axis="columns")
)

decisiveness_table = pd.pivot(
    decision_counts.groupby(["dataset", "model", "task_scope"])["decisiveness"].mean().reset_index(),
    index="dataset",
    columns=["model", "task_scope"],
    values="decisiveness",
)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(
    data=go.Heatmap(
        x=(decisiveness_table.columns.get_level_values(0), decisiveness_table.columns.get_level_values(1)),
        y=decisiveness_table.index,
        z=decisiveness_table.values,
        text=decisiveness_table.values,
        texttemplate="%{text:.3f}",
        textfont={"size": 16},
        colorscale="PrGn",
        zmin=-1.0,
        zmax=1.0,
        showscale=False,
    ),
    layout=dict(
        title="Average fraction of non-unknown decisions per task scope",
        height=600,
        width=1000,
        yaxis={"autorange": "reversed"},
    ),
)
fig.show()